In [ ]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
export OS_PROJECT_NAME='CH-816532'

# Set region (again, no default currently)
export OS_REGION_NAME='CHI@TACC'

# Set chameleon keypair name and path to the private ssh key
export SSH_KEY_NAME=${USERNAME}-jupyter
export SSH_PRIVATE_KEY=${HOME}/work/pruth-chameleon-jupyter


#Set the working directory
export WORKING_DIR=${HOME}/work/CICI-SAFE/Chameleon-Jupyter-Tridentcom

export UNIVERSAL_INFO_FILE=${WORKING_DIR}/universal_info
echo "" > $UNIVERSAL_INFO_FILE



#### Create Lease (if necessary)

In [ ]:
# --start-date "2019-02-10 17:50" \
# --end-date "2019-02-11 12:00" \
START_DATE=`date -d "+2 min" +'%F %H:%M'`
END_DATE=`date -d "+1 day" +'%F %H:%M'`
LEASE_NAME=${USERNAME}"-cici-universal-lease"
echo LEASE_NAME $LEASE_NAME >> $UNIVERSAL_INFO_FILE

NODE_MIN=4
NODE_MAX=6

NODE_TYPE='compute_haswell'
#NODE_TYPE='compute_skylake'

echo Creating Lease ${LEASE_NAME}
blazar lease-create \
   --physical-reservation min=${NODE_MIN},max=${NODE_MAX},resource_properties='["=", "$node_type", "'${NODE_TYPE}'"]' \
   --start-date "${START_DATE}" \
   --end-date "${END_DATE}" \
   ${LEASE_NAME}
   

In [ ]:
RESERVATION_ID=`blazar lease-show --format value -c id ${LEASE_NAME}`
echo RESERVATION_ID $RESERVATION_ID

RESERVATION_RESOURCE_ID=`blazar lease-show --format value -c reservations ${RESERVATION_ID} | grep \"id\" | cut -d "\"" -f 4`
echo RESERVATION_RESOURCE_ID $RESERVATION_RESOURCE_ID

while [[ true ]]
do 
	RESERVATION_STATUS=`blazar lease-show --format value -c status ${RESERVATION_ID}`
    
    if [[ "${RESERVATION_STATUS}" == "ACTIVE" ]]; then break; fi
    
    echo Reservation is ${RESERVATION_STATUS}. retrying in 5 seconds.
    sleep 5
done

echo RESERVATION_STATUS ${RESERVATION_STATUS}

echo RESERVATION_RESOURCE_ID $RESERVATION_RESOURCE_ID >> $UNIVERSAL_INFO_FILE

cat $UNIVERSAL_INFO_FILE


#### Clean Up

In [ ]:
LEASE_NAME=`cat $UNIVERSAL_INFO_FILE | grep LEASE_NAME | cut -d " " -f 2`
blazar lease-delete $LEASE_NAME

cat $UNIVERSAL_INFO_FILE
mv $UNIVERSAL_INFO_FILE ${UNIVERSAL_INFO_FILE}.old